In [52]:
import gurobipy as gp
from gurobipy import GRB

# The Model

In [53]:
theater_seat_organizer = gp.Model()

# The Parameters

In [95]:
# Uncomment the single file that will be tested and recomment the previous case

test_case = "testcase1-poly.txt"
#test_case = "testcase2-poly.txt"
#test_case = "testcase3-poly.txt"
#test_case = "testcase4-poly.txt"

f = open(test_case,"r")
testfile = f.readlines()

test_name = testfile[0].split("=")[1]

string_num_of_customers_in_group = testfile[1].split(" = ")[1]

#Convert list of string to list of int
number_of_customers_in_group_list = string_num_of_customers_in_group.strip("[").strip("\n").strip("]").split(",")
map_list = map(int, number_of_customers_in_group_list)
number_of_customers_in_group = list(map_list)

number_of_seats = int(testfile[2].split(" = ")[1])

group_bonus = int(testfile[3].split(" = ")[1])

# Decision Variables

In [96]:
person_inGroup_atSeat = {}

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            
            person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))


# The Model

In [97]:
objectiveFunction = gp.LinExpr()

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   
    
for group_num in range(len(number_of_customers_in_group)):
    
    objectiveFunction += 

theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)

# Constraints

In [98]:
# at most one person per seat
for seat in range(number_of_seats):
    at_most_one_person_per_seat = gp.LinExpr()
    
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            at_most_one_person_per_seat += person_inGroup_atSeat[customer_index, group_num, seat]   
    theater_seat_organizer.addConstr(at_most_one_person_per_seat <= 1, name = "at_most_one_person_at_seat" + str(seat))



In [100]:
# each person should be assigned to only one seat 
for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        seats_per_person = gp.LinExpr()
        
        for seat in range(number_of_seats):
            seats_per_person += person_inGroup_atSeat[customer_index, group_num, seat] 
            
        theater_seat_organizer.addConstr(seats_per_person <= 1, name = "person" + str(customer_index) + "_group" + str(group_num) + "_uniqueSeat")


In [103]:
# There must be an empty seat between groups 
for seat in range(number_of_seats - 1):
    
    for group_num in range(len(number_of_customers_in_group)):
        for other_group_num in range(len(number_of_customers_in_group)):
            
            if not(group_num == other_group_num):    
                
                for customer_index in range(0, number_of_customers_in_group[group_num]):
                    for customer2_index in range(0, number_of_customers_in_group[other_group_num]):
                        
                        seat1 = person_inGroup_atSeat[customer_index, group_num, seat]
                        seat2 = person_inGroup_atSeat[customer2_index, other_group_num, seat+1]
                        label = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_andPerson" + str(customer2_index) + "+inGroup" + str(other_group_num) + "_cannotSitTogether"
                        theater_seat_organizer.addConstr( seat1 + seat2 <= 1, name = label)

# Optimize 

In [102]:
theater_seat_organizer.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2674786 rows, 25302 columns and 5423211 nonzeros
Model fingerprint: 0x7f5f2e4a
Variable types: 0 continuous, 25302 integer (25302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 108.96 (100.11s)
MIP start from previous solve produced solution with objective 113.97 (114.65s)
MIP start from previous solve produced solution with objective 117.97 (474.14s)
MIP start from previous solve produced solution with objective 121.98 (541.30s)
MIP start from previous solve produced solution with objective 123.99 (740.01s)
MIP start from previous solve produced solution with objective 126.99 (2112.44s)
MIP start from previous solve produced solution with objective 127.

# Post Process

In [107]:
seating_scheme = list()
for seat in range(number_of_seats):
    seatFound = False
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            result = person_inGroup_atSeat[customer_index, group_num, seat]
            
            if( result.x == 1 ):
                seatFound = True
                print( "Seat" + str(seat) + ": person" + str(customer_index) + " fromGroup" + str(group_num))
                
                seating_scheme.append((customer_index, group_num))
    
    if(not seatFound):
        print("Seat" + str(seat) + ": <empty>")
        seating_scheme.append(None)

Seat0: person5 fromGroup13
Seat1: person0 fromGroup13
Seat2: person3 fromGroup13
Seat3: person4 fromGroup13
Seat4: <empty>
Seat5: person2 fromGroup17
Seat6: person0 fromGroup17
Seat7: person1 fromGroup17
Seat8: person3 fromGroup17
Seat9: <empty>
Seat10: person1 fromGroup5
Seat11: person3 fromGroup5
Seat12: person4 fromGroup5
Seat13: person5 fromGroup5
Seat14: person2 fromGroup5
Seat15: person0 fromGroup5
Seat16: <empty>
Seat17: person0 fromGroup15
Seat18: person6 fromGroup15
Seat19: person2 fromGroup15
Seat20: person5 fromGroup15
Seat21: <empty>
Seat22: person7 fromGroup14
Seat23: person0 fromGroup14
Seat24: person1 fromGroup14
Seat25: <empty>
Seat26: person7 fromGroup20
Seat27: person6 fromGroup20
Seat28: person8 fromGroup20
Seat29: person3 fromGroup20
Seat30: person4 fromGroup20
Seat31: person1 fromGroup20
Seat32: person0 fromGroup20
Seat33: <empty>
Seat34: person1 fromGroup13
Seat35: <empty>
Seat36: person2 fromGroup12
Seat37: person5 fromGroup12
Seat38: person3 fromGroup12
Seat39: 

In [105]:
genericSpacing = list()
last_group = -1
for value in seating_scheme:
    if(type(value) == tuple and value[1] != last_group):
        genericSpacing.append((None,None))
        genericSpacing.append(value)
        last_group = value[1]
        
    
    elif(type(value) == tuple and value[1] == last_group):
        genericSpacing.append(value)
        
genericSpacing.pop(0)

while(genericSpacing[0][1] == genericSpacing[-1][1]):
    genericSpacing.append(genericSpacing[0])
    genericSpacing.pop(0)
    
genericSpacing.pop(0)
    
for members in genericSpacing:
    if not (members[0] == None):
        print("person " + str(members[0] + 1) + " from group " + str(members[1] + 1))
    else:
        print("<SPACE>")



person 1 from group 14
person 4 from group 14
person 5 from group 14
<SPACE>
person 3 from group 18
person 1 from group 18
person 2 from group 18
person 4 from group 18
<SPACE>
person 2 from group 6
person 4 from group 6
person 5 from group 6
person 6 from group 6
person 3 from group 6
person 1 from group 6
<SPACE>
person 1 from group 16
person 7 from group 16
person 3 from group 16
person 6 from group 16
<SPACE>
person 8 from group 15
person 1 from group 15
person 2 from group 15
<SPACE>
person 8 from group 21
person 7 from group 21
person 9 from group 21
person 4 from group 21
person 5 from group 21
person 2 from group 21
person 1 from group 21
<SPACE>
person 2 from group 14
<SPACE>
person 3 from group 13
person 6 from group 13
person 4 from group 13
person 7 from group 13
person 1 from group 13
person 5 from group 13
person 2 from group 13
<SPACE>
person 4 from group 8
person 2 from group 8
person 1 from group 8
person 5 from group 8
person 3 from group 8
person 8 from group 8
perso

In [108]:
for group_num in range(len(groupAccepted)):
    if(groupAccepted[group_num].x != 1):
        print("Group " + str(group_num + 1) + " is excluded")
    else:
        print("Group " + str(group_num + 1) + " is accepted")
#print(groupAccepted)

Group 1 is accepted
Group 2 is accepted
Group 3 is accepted
Group 4 is accepted
Group 5 is accepted
Group 6 is accepted
Group 7 is accepted
Group 8 is accepted
Group 9 is accepted
Group 10 is accepted
Group 11 is accepted
Group 12 is accepted
Group 13 is accepted
Group 14 is accepted
Group 15 is accepted
Group 16 is accepted
Group 17 is accepted
Group 18 is accepted
Group 19 is accepted
Group 20 is accepted
Group 21 is accepted
